# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np
from sklearn import preprocessing,svm
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, mean_absolute_error, mean_squared_error,f1_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, Normalizer
from itertools import product, combinations
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import VotingClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split
from sklearn.feature_selection import mutual_info_classif,SelectFromModel
from sklearn.decomposition import PCA
import itertools
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import cvxopt
import matplotlib.mlab as mlab
import seaborn
import pickle


%matplotlib inline

In [ ]:
combined_features='/content/drive/MyDrive/Capstone/Graph properties /Global feature files/Global combined.csv'
binary_features= '/content/drive/MyDrive/Capstone/Graph properties /Global feature files/global_combined_binary_cleaned.csv'
weighted_features='/content/drive/MyDrive/Capstone/Graph properties /Global feature files/global_combined_weighted.csv'
path=binary_features

In [ ]:
df= pd.read_csv(path)
#df=df.drop(['Unnamed: 0','Randomized Partitioning Heuristic'], axis=1)
df

,Unnamed: 0,Subject,Approx Node Connectivity,Approx Max Independent Sets,Approx Large Clique Size,Approx Average Clustering,Approx Diameter,Approx Treewidth Min Degree,Degree Assortativity,Avg Shortest Path Length,...,Avg Degree,Global Clustering,Edges,Maximal Cliques,Non Randomness wrt Random Model,Transitivity,Connected Components,Diameter,3 Edge Connected,Schizophrenic
0,0,2034,17,319,30,0.578,3,132,0.197773,1.593371,...,33.597561,0.602853,5510,30092,10.898755,0.602900,1,3,1,0
1,1,2056,23,319,25,0.520,3,120,0.221608,1.686892,...,26.335366,0.536917,4319,6608,6.660088,0.535128,1,3,1,0
2,2,2057,17,319,21,0.556,3,119,0.179582,1.684423,...,26.707317,0.537487,4380,8727,4.137419,0.528799,1,3,1,0
3,3,2024,20,319,24,0.538,3,131,0.249837,1.703501,...,25.006098,0.520660,4101,6717,5.797461,0.526791,1,3,1,0
4,4,2050,18,319,23,0.524,3,135,0.359868,1.697292,...,25.841463,0.522355,4238,8022,8.437650,0.534831,1,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,314,40141,19,319,25,0.556,3,121,0.295640,1.710310,...,25.829268,0.549265,4236,9404,12.395470,0.554442,1,3,1,0
315,315,40123,6,319,28,0.580,3,107,0.438202,1.772183,...,24.323171,0.584091,3989,4212,-5.458323,0.619409,1,3,1,0
316,316,40116,7,319,29,0.613,3,107,0.252487,1.846551,...,21.957317,0.601012,3601,2212,6.571716,0.615655,1,3,1,0
317,317,40115,13,319,18,0.530,3,111,0.323508,1.818794,...,20.780488,0.518191,3408,4311,10.225171,0.522160,1,3,1,0


In [ ]:
#UCLA Subjects
df_filtered=df[df['Subject'] < 30000]
df=df_filtered

# Cleaing and preprocessing 

In [ ]:
non_unique=list()
for column in df:
    if(len(pd.unique(df[column]))==1):
        non_unique.append(column)

non_unique

['Approx Max Independent Sets',
 'Bridges',
 'Connected Components',
 'Diameter',
 '3 Edge Connected']

In [ ]:
df=df.drop(non_unique, axis=1)

In [ ]:
X = df.drop('Schizophrenic', axis=1)
X = X.drop('Subject', axis=1)
y = df['Schizophrenic']

In [ ]:
pipe = Pipeline([('standardscaler',StandardScaler()),('norm',Normalizer())])
model = pipe.fit(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# ML Models

In [ ]:
#best depth: max_depth=6
clf1 = DecisionTreeClassifier(criterion="entropy", class_weight={0:122, 1:50})
#best k -> 2 
clf2 = KNeighborsClassifier(n_neighbors=3 )
clf3 = SVC(kernel="poly", degree=4,class_weight={0:122, 1:50})
clf4=LogisticRegression(class_weight={0:122, 1:50})
clf5=RandomForestClassifier(n_estimators=500, criterion="entropy", max_features="log2", class_weight={0:122, 1:50})
clf6=GradientBoostingClassifier(n_estimators = 300, max_features="log2", max_depth=7)
clf7=AdaBoostClassifier(n_estimators=200)

clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)
clf4.fit(X_train, y_train)
clf5.fit(X_train, y_train)
clf6.fit(X_train, y_train)
clf7.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=200)

In [ ]:
pred = clf2.predict(X_test)
print("KNN accuracy:", accuracy_score(y_test, pred)*100)
print(confusion_matrix(y_test, pred))
print("f1 score: ", f1_score(y_test, pred))

KNN accuracy: 71.42857142857143
[[23  3]
 [ 7  2]]
f1 score:  0.2857142857142857


In [ ]:
pred = clf3.predict(X_test)
print("SVM accuracy:", accuracy_score(y_test, pred)*100)
print(confusion_matrix(y_test, pred))
print("f1 score: ", f1_score(y_test, pred))

SVM accuracy: 71.42857142857143
[[25  1]
 [ 9  0]]
f1 score:  0.0


In [ ]:
pred = clf4.predict(X_test)
print("Logistic Regression accuracy:", accuracy_score(y_test, pred)*100)
print(confusion_matrix(y_test, pred))
print("f1 score: ", f1_score(y_test, pred))

Logistic Regression accuracy: 82.85714285714286
[[24  2]
 [ 4  5]]
f1 score:  0.6250000000000001


In [ ]:
pred = clf1.predict(X_test)
print("Decision tree accuracy:", accuracy_score(y_test, pred)*100)
print(confusion_matrix(y_test, pred))
print("f1 score: ", f1_score(y_test, pred))

Decision tree accuracy: 88.57142857142857
[[23  3]
 [ 1  8]]
f1 score:  0.7999999999999999


In [ ]:
pred = clf7.predict(X_test)
print("AdaBoost :", accuracy_score(y_test, pred)*100)
print(confusion_matrix(y_test, pred))
print("f1 score: ", f1_score(y_test, pred))

AdaBoost : 85.71428571428571
[[24  2]
 [ 3  6]]
f1 score:  0.7058823529411765
